# Reponses from the Webserver 

`DFObserve` contains a few wrapper classes that make interacting with the output of a WebRequest more functional. Generally these are used "under the hood," but let's go over how it works for completeness (as well as for adding new functions and features). 

In [1]:
from dfobserve.webserver import SendWebRequestNB

We've imported the primary SendWebRequest function. Let's ask for a 'status' from the server:

In [2]:
status = SendWebRequestNB('status')

The following commands are queued to send to the following units
----------------------------------------------------------------
           Name             ip command
0  Dragonfly301  192.168.50.11  status
1  Dragonfly302  192.168.50.12  status
2  Dragonfly303  192.168.50.13  status
3  Dragonfly304  192.168.50.14  status
4  Dragonfly305  192.168.50.15  status
5  Dragonfly306  192.168.50.16  status
6  Dragonfly307  192.168.50.17  status
7  Dragonfly308  192.168.50.18  status
8  Dragonfly309  192.168.50.19  status
9  Dragonfly310  192.168.50.20  status
Sending Request...


Initial Responses:
------------------
           Name             ip command        response_summary  \
0  Dragonfly301  192.168.50.11  status                 SUCCESS   
1  Dragonfly302  192.168.50.12  status  Machine Down (URL err)   
2  Dragonfly303  192.168.50.13  status                 SUCCESS   
3  Dragonfly304  192.168.50.14  status                 SUCCESS   
4  Dragonfly305  192.168.50.15  status              

We can see the verbose reponse from the instrument above. It sent the status command to all IPs, and then we got responses, which in this case were 9 successes and 1 machine down. Notice in the `full_response` column, the successful responses contain `APIResponse` objects. Let's look at one.

To do so, we need to retrieve it from the `status` object, which is an instance of `WebRequestSummary`. We can do this easily either by dot notation and the unit name, or using the `get_response()` method which takes in the desired IP address.

In [3]:
df301 = status.Dragonfly301
df301

APIResponse[192.168.50.11]

The `APIResponse` object has a handy `info()` method which can tell us its attributes:

In [4]:
df301.info()

Status Response for Unit at IP: 192.168.50.11 
 Accessible Fields: 
 ['Activity', 'CameraProperties', 'CurrentExposure', 'Focus', 'LastExposure', 'XtraCalculations', 'FlipFlat', 'FilterTilter']


So we can now access, say, `CameraProperties` by dot notation:

In [5]:
df301.CameraProperties

                                          0
ExposeErrorHasOccurred                False
CurrentDirectory                      /data
NumberOfImagesTaken                       0
FocusModel              21763-10.0*(x-54.0)
ReadNoise                                -1
Bias                                     -1
FilterName                          Unknown
SerialNumber                        Unknown
IPAddress                     192.168.50.11
CurrentTemperature                     None
SetpointTemperature                    None
AmbientTemperature                     None
Power                                  None
FocuserSerialNumber                 Unknown
LensSerialNumber                    Unknown

Each field (e.g., `CameraProperties`) is also a wrapped object (technically, the `WrapDF` object), meaning we can once again dot-notate to get a value out:

In [6]:
df301.CameraProperties.FocusModel

'21763-10.0*(x-54.0)'

For completeness, there is also an explicit `get()` method, e.g., 

In [7]:
df301.CameraProperties.get('FocusModel')

'21763-10.0*(x-54.0)'

Finally, the `APIResponse.info()` method introduced above has a verbose mode that will print the fields AND their internals, if you want to see everything all at once:

In [8]:
df301.info(verbose=True)

Status Response for Unit at IP: 192.168.50.11

 

ATTRIBUTE: Activity
                                    0
Any                             False
ExposureInProgress              False
BirgerCommandInProgress         False
FocusRunInProgress              False
CalculationInProgress           False
CameraCheckInProgress           False
RegulationSettingInProgress     False
FlipFlatActivityInProgress       None
FilterTilterActivityInProgress   None
FilterTilterCheckInProgress     False

 

ATTRIBUTE: CameraProperties
                                          0
ExposeErrorHasOccurred                False
CurrentDirectory                      /data
NumberOfImagesTaken                       0
FocusModel              21763-10.0*(x-54.0)
ReadNoise                                -1
Bias                                     -1
FilterName                          Unknown
SerialNumber                        Unknown
IPAddress                     192.168.50.11
CurrentTemperature                     N

You may notice that all of the printouts of these objects appear as pandas DataFrames. Currently, `SendWebRequestNB` constructs DataFrames to send, and parses responses into DataFrames as well. The raw DataFrames can be accessed in either a `WebRequestSummary` object or in an `APIResponse` object or in a `WrapDF` object via the `df` attribute. This is not strictly speaking useful unless you prefer to work with a DataFrame for some particular reason. Note that the *attributes* of each object are read off of the DataFrame upon instantiation, and currently, do not support @property tags to read them dynamically from the DataFrame. Hence, one can modify values in the DataFrame and make them inconsistent with the values in the same-named attribute. 

I plan to fix this in the future but for now it is not a huge concern because *responses* from the Webserver should be read-only in practice anyway. At least at the initial response layer.  